In [208]:
import numpy as np
import pandas as pd
import time

In [219]:
prod_filepath = "./data/production_data.csv"
df_prod = pd.read_csv(prod_filepath)

alias_filepath = "./data/operator_aliasing.csv"
df_alias = pd.read_csv(alias_filepath)

wells_filepath = "./data/wellsData.csv"
df_wells = pd.read_csv(wells_filepath)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Adjusting API by adding a leading zero to APIs starting with '5'

In [220]:
start = time.time()
df_prod.API=df_prod.API.astype(str).str.zfill(14)
df_wells.API=df_wells.API.astype(str).str.zfill(14)

Add Operator Alias

In [224]:
filter_col = np.append(df_prod.columns, ['Alias'])

# wells_alias_df = df_alias[['Name', 'Alias']].merge(df_wells[['operatorNameIHS', 'API']], left_on='Name', right_on='operatorNameIHS', how='outer')
new_df = (df_prod.merge(df_alias[['Name', 'Alias']].merge(df_wells[['operatorNameIHS', 'API']], left_on='Name', right_on='operatorNameIHS', how='right')[['operatorNameIHS', 'Alias', 'API']], on='API', how='left'))

new_df = new_df.fillna({'Alias':new_df.operatorNameIHS})[filter_col]

,API,Year,Month,Liquid,Gas,RatioGasOil,Water,PercentWater,DaysOn,_LastUpdate,Alias
0,35045231810000,2012,8,0,9014,0.000000,0,0.000000,0,2016-04-06 09:53:22.573,P V PROPERTIES
1,35045231810000,2012,9,0,39672,0.000000,0,0.000000,0,2016-04-06 09:53:22.573,P V PROPERTIES
2,35045231810000,2012,10,0,21454,0.000000,0,0.000000,0,2016-04-06 09:53:22.573,P V PROPERTIES
3,35045231810000,2012,11,177,6259,35.361582,0,0.000000,0,2016-04-06 09:53:22.573,P V PROPERTIES
4,35045231810000,2012,12,0,10666,0.000000,0,0.000000,0,2016-04-06 09:53:22.573,P V PROPERTIES
5,35045231810000,2013,1,0,1500,0.000000,0,0.000000,0,2016-04-06 09:53:22.573,P V PROPERTIES
6,33089006470100,2011,12,0,0,0.000000,10454,0.000000,31,2016-04-06 09:53:24.683,CHESAPEAKE
7,33089006470100,2012,1,0,0,0.000000,10982,0.000000,0,2016-04-06 09:53:24.683,CHESAPEAKE
8,35029211390000,2012,2,0,58971,0.000000,0,0.000000,0,2016-04-06 09:53:55.693,NEWFIELD
9,35029211390000,2012,3,0,0,0.000000,0,0.000000,0,2016-04-06 09:53:55.693,NEWFIELD


Remove Pre-Peak Months

In [225]:
# filtering out post-peak records with 0 oil production
# tmp_df = new_df[filter_col][new_df['Liquid']!=0]

# getting peak records
peak_records = new_df[new_df.groupby(['API'], sort=False)['Liquid'].transform(max) == new_df['Liquid']]

# filtering out pre-peak months
peak_records = peak_records.rename(columns={"Month": "PeakMonth", "Year": "PeakYear"})
new_df = pd.merge(new_df, peak_records[["API", "PeakMonth", "PeakYear"]], on='API', how='left')
post_peak = new_df[np.logical_or((new_df['Year'] > new_df['PeakYear']),np.logical_and((new_df['Year']==new_df['PeakYear']),(new_df['Month']>=new_df['PeakMonth'])))].sort_values(['Year','Month'])[filter_col]




Add a Month Index Column

In [226]:
post_peak['MonthIdx'] = post_peak.groupby(['API'], sort=False).cumcount()+1
end = time.time()
tt = end - start
print(tt)

123.15779709815979


In [ ]:
post_peak[filter_col].to_csv("./data/processed_production_data.csv")